In [1]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

在這裡我們先定義一個函數，希望把網頁中各篇貼文的連結都找出來!
ulr放我們要爬的Facebook網址，n是稍後要送出幾次滾動網頁到底部的命令，藉以加載更多資料。

In [2]:
def FindLinks(url, n):
    Links = []
    driver.get(url)
    for i in range(n):
        time.sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    for i in posts:
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
    return Links

定義一個展開所有留言的函數，透過while迴圈反覆搜尋與點擊「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕。
在過程中會出現請我們登入或註冊的彈跳視窗，但我們不確定到底什麼時候會跳出，因此需要在過程中反覆偵測是有出現這個彈窗，若有就點擊「Not Now」

In [3]:
def expand(url):
    driver.get(url)
#    time.sleep(2)
#    try:
#        driver.find_element_by_xpath('//a[@lang="en_US"]').click()
#    except:
#        print("Now is in EN_US")
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # 點擊「comments」，藉以展開留言
    try:
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.find_element_by_id('expanding_cta_close_button').click() 
    except:
        print('There is no comment!')
    k = 1
    while k != 0:
        k = 0
        for i in driver.find_elements_by_xpath('//div[@class="_5pcr userContentWrapper"]//div[@data-testid="UFI2CommentsList/root_depth_0"]//a[@role="button"]'): 
            # 反覆偵測是否有「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕，若有擇點擊
            if bool(re.search('comment|More|Repl',i.text)) == True :
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                try:
                    driver.find_element_by_xpath('//div[@style="display: block;"]//a[@id="expanding_cta_close_button"]').click()
                except:
                    time.sleep(0.5)
                try:
                    i.click()
                except:
                    time.sleep(0.5)
                time.sleep(1)
                k += 1

透過逐層搜索的方式，逐步定位我們要找的資訊
在這個環節需要反覆透過Chrome的功能比對資料，需要花一些心力進行比對
另外在這部分也使用了大量個try-except，原因是許多資料是有內容才會出現。例如並非每天貼人都會收到「哈哈」、「生氣」的心情。

In [4]:
# 文章內容與互動摘要
def PostContent(soup):
    time.sleep(1)
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
    # 互動摘要區(讚、留言與分享)
    feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = PosterInfo.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?',2)[0].split('/',-1)[-1]
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'}).text
    # 文章內容
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = ""
    # Like
    try:
        Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label']
    except:
        Like = '0 Like' 
    # Love
    try:
        Love = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label']
    except:
        Love = '0 Love' 
    # Haha
    try:
        Haha = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label']
    except:
        Haha = '0 Haha' 
    # Wow
    try:
        Wow = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label']
    except:
        Wow = '0 Wow' 
    # Sad
    try:
        Sad = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label']
    except:
        Sad = '0 Sad' 
    # Angry
    try:
        Angry = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label']
    except:
        Angry = '0 Angry'
    # 留言
    try:
        commentcount = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text
    except:
        commentcount = '0' 
    # 分享
    try:
        Share = feedback.find('span', {'class':'_355t _4vn2'}).text
    except:
        Share = '0' 
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content,
                 'Like':Like,
                 'Love':Love,
                 'Haha':Haha,
                 'Wow':Wow,
                 'Sad':Sad,
                 'Angry':Angry,
                 'commentcount':commentcount,
                 'Share':Share}],
        columns = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'commentcount', 'Share', 'Link'])

這邊要留意雖然都是粉絲留言，但實際上分成「回應貼文的留言」與「回應留言的留言」。
函數中的第一個迴圈是用來抓「回應貼文的留言」，第二個則是抓「回應留言的留言」。讀者可以自行比較一下兩個迴圈中不同的地方。

In [5]:
# 留言
def CrawlComment(soup):
    time.sleep(1)
    Comments = pd.DataFrame()
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 用戶留言區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 回應貼文的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_0'}):
        try:
            CommentContent = i.find('span', {'dir':'ltr'}).text
        except:
            CommentContent = 'Sticker'
        Comment = pd.DataFrame(data = [{'CommentID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'CommentName':i.find('img').attrs['alt'],
                                 'CommentTime':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'CommentContent':CommentContent,
                                 'Link':driver.current_url}],
                        columns = ['CommentID', 'CommentName', 'CommentTime', 'CommentContent', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    
    # 回應留言的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
        try:
            CommentContent = i.find('span', {'dir':'ltr'}).text
        except:
            CommentContent = 'Sticker'
        Comment = pd.DataFrame(data = [{'CommentID':i.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'CommentName':i.find('img').attrs['alt'],
                                 'CommentTime':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'CommentContent':CommentContent,
                                 'Link':driver.current_url}],
                        columns = ['CommentID', 'CommentName', 'CommentTime', 'CommentContent', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)        
    return Comments

In [6]:
# https://www.facebook.com/cathayunitedbank/
# https://www.facebook.com/tstartel/
# https://www.facebook.com/chtmobile/
# https://www.facebook.com/taiwanmobile/
# https://www.facebook.com/fareastone/
# https://www.facebook.com/Aptg.tw/
driver = webdriver.Chrome()
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath('//a[@class="_sv4"]').click()

In [7]:
url = 'https://www.facebook.com/tstartel/'
Links = FindLinks(url = url, n = 5)
Links

['https://www.facebook.com/tstartel/posts/3251586194867405',
 'https://www.facebook.com/tstartel/photos/a.2512681442091221/3249335421759149/',
 'https://www.facebook.com/tstartel/photos/a.413576055335114/3246516712041020/',
 'https://www.facebook.com/tstartel/photos/a.413576055335114/3243531712339520/',
 'https://www.facebook.com/tstartel/photos/a.854369967922385/3243588789000479/',
 'https://www.facebook.com/tstartel/photos/a.2512681442091221/3241649495861075/',
 'https://www.facebook.com/tstartel/photos/a.413576055335114/3238717112820980/',
 'https://www.facebook.com/tstartel/photos/a.413576055335114/3236285659730792/',
 'https://www.facebook.com/tstartel/photos/a.413576055335114/3234262259933132/',
 'https://www.facebook.com/tstartel/photos/a.2512681442091221/3234388456587179/',
 'https://www.facebook.com/tstartel/videos/356966824846946/',
 'https://www.facebook.com/tstartel/posts/3231419526884072',
 'https://www.facebook.com/tstartel/photos/a.854369967922385/3229153237110701/',
 'h

In [39]:
# 抓下來所有留言
PostsInformation = pd.DataFrame()
PostsComments = pd.DataFrame()
for i in Links:
    print('Dealing with: ' + i)
    try:
        expand(i)
        soup = BeautifulSoup(driver.page_source)
        PostsInformation = pd.concat([PostsInformation, PostContent(soup)],ignore_index=True)
        PostsComments = pd.concat([PostsComments, CrawlComment(soup)],ignore_index=True)
        print('Load Succed: ' + i)
    except:
        print('Load Failed: ' + i)

Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3238717112820980/
Load Succed: https://www.facebook.com/tstartel/photos/a.413576055335114/3238717112820980/
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3236285659730792/
There is no comment!
Load Succed: https://www.facebook.com/tstartel/photos/a.413576055335114/3236285659730792/
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3234262259933132/
Load Succed: https://www.facebook.com/tstartel/photos/a.413576055335114/3234262259933132/
Dealing with: https://www.facebook.com/tstartel/photos/a.2512681442091221/3234388456587179/
There is no comment!
Load Succed: https://www.facebook.com/tstartel/photos/a.2512681442091221/3234388456587179/
Dealing with: https://www.facebook.com/tstartel/videos/356966824846946/
There is no comment!
Load Succed: https://www.facebook.com/tstartel/videos/356966824846946/
Dealing with: https://www.facebook.com/tstartel/posts/32314195268840

In [40]:
PostsInformation

,Name,ID,Time,Content,Like,Love,Haha,Wow,Sad,Angry,commentcount,Share,Link
0,台灣之星,tstartel,"5/1/19, 9:25 PM",#送雙人饗食天堂假日下午餐券 小編團們苦思母親節活動還在難產時 #週星星編 開了媽媽第一槍後...,419 Like,4 Love,21 Haha,0 Wow,0 Sad,0 Angry,240 Comments,208 Shares,https://www.facebook.com/tstartel/photos/a.413...
1,台灣之星,tstartel,"4/30/19, 8:24 PM",現在的小孩真的都好聰明像個小大人😂 #MrX編 今天終於可以忙裡偷閒 趁著 #51勞動節 在...,510 Like,26 Love,36 Haha,0 Wow,0 Sad,0 Angry,55 Comments,12 Shares,https://www.facebook.com/tstartel/photos/a.413...
2,台灣之星,tstartel,"4/30/19, 5:05 AM",明天放一天假，做二休一再做二休二 這樣的工作節奏 #U姐編 覺得很可以 但是想到五月要繳稅💸...,718 Like,0 Love,0 Haha,16 Wow,0 Sad,9 Angry,442 Comments,23 Shares,https://www.facebook.com/tstartel/photos/a.413...
3,台灣之星,tstartel,"4/30/19, 2:06 AM",走過三十個年頭 日本 #平成 #年號 即將在今天畫下句點 緊接而來的 #令和 時代 又會為日...,515 Like,13 Love,0 Haha,2 Wow,0 Sad,0 Angry,60 Comments,11 Shares,https://www.facebook.com/tstartel/photos/a.251...
4,台灣之星,tstartel,"4/29/19, 1:52 AM",#eSIM全新上線 #網路門市也能辦 ➡http://bit.ly/2GzAoW5 📣iPh...,150 Like,1 Love,0 Haha,0 Wow,3 Sad,0 Angry,95 Comments,12 Shares,https://www.facebook.com/tstartel/videos/35696...
5,台灣之星,tstartel,"4/28/19, 9:00 PM",後天是 #勞動節，雖然賺到一天假 但接著就要繳稅吃土一個月😱😱😱 #憤輕編 懷疑這根本就是安...,686 Like,4 Love,7 Haha,0 Wow,0 Sad,0 Angry,510 Comments,253 Shares,https://www.facebook.com/tstartel/posts/323141...
6,台灣之星,tstartel,"4/27/19, 9:22 PM",#週1福利日 #每週一1230 #週週登場 聰明的你不用 #MrX編 週週提醒😬 把網址加入...,391 Like,4 Love,0 Haha,2 Wow,0 Sad,0 Angry,90 Comments,6 Shares,https://www.facebook.com/tstartel/photos/a.854...
7,台灣之星,tstartel,"4/26/19, 9:00 PM",#憤輕編 眼睛要花了啦 這根本是催眠圖片啊😴哪裡有數字喇🤨 大家都有看到數字嗎？留言告訴小編...,280 Like,0 Love,2 Haha,9 Wow,0 Sad,0 Angry,207 Comments,18 Shares,https://www.facebook.com/tstartel/posts/322466...
8,台灣之星,tstartel,"4/26/19, 12:23 AM",我的天啊～#00C編 的Google怎麼壞掉了 怎麼好像 #復仇者聯盟 的場景啊 畫面竟然全...,236 Like,0 Love,17 Haha,5 Wow,0 Sad,0 Angry,28 Comments,5 Shares,https://www.facebook.com/tstartel/photos/a.413...
9,台灣之星,tstartel,"4/25/19, 2:21 AM",滴答滴答～什麼！竟然過了三個小時😨 #00C編 明明只是滑個FB啊...... 嘿嘿🤭是不是...,314 Like,4 Love,0 Haha,0 Wow,0 Sad,2 Angry,14 Comments,4 Shares,https://www.facebook.com/tstartel/photos/a.854...


In [41]:
PostsComments

,CommentID,CommentName,CommentTime,CommentContent,Link
0,360044337354953,台灣之星,"Thursday, May 2, 2019 at 12:26 PM",活動時間： 即日起至5/9(四)23:59止 活動獎項： 雙人饗食天堂假日下午餐券x5名 活...,https://www.facebook.com/tstartel/photos/a.413...
1,100007904763492,吳許許,"Thursday, May 2, 2019 at 12:28 PM",蔡香包，叫我早點上床睡覺，沒多久又叫我起來吃消夜，到底！,https://www.facebook.com/tstartel/photos/a.413...
2,100002039591055,劉佩盈,"Thursday, May 2, 2019 at 12:30 PM",黃忻诶 我媽媽問我眼鏡在哪，請我幫忙找，結果在她臉上😂,https://www.facebook.com/tstartel/photos/a.413...
3,100000156002617,吳兆文,"Thursday, May 2, 2019 at 12:30 PM",殷辰暟，叫我早點上床睡覺，沒多久又叫我起來吃消夜，到底！,https://www.facebook.com/tstartel/photos/a.413...
4,100000222771299,陳小霞,"Thursday, May 2, 2019 at 12:32 PM",李震宇 叫我減肥...卻買了滿桌的食物，還準備一堆東西叫我帶回家。😁,https://www.facebook.com/tstartel/photos/a.413...
5,100013027844179,Hanyii Chiu,"Thursday, May 2, 2019 at 12:33 PM",黃明蓉 （Li Hung），用煮飯鍋煮了一大盆燙青菜叫我吃，還說吃菜不會飽 (吃菜不會飽，那...,https://www.facebook.com/tstartel/photos/a.413...
6,100015267528767,Tristy Hsiao,"Thursday, May 2, 2019 at 12:33 PM",Emma Chu ，每天早上都要問我長輩圖傳那張好，那張比較有意義，我看起來都一樣啊啊啊啊啊！,https://www.facebook.com/tstartel/photos/a.413...
7,100002770694788,劉柏成,"Thursday, May 2, 2019 at 12:34 PM",Ya Halo Yen，媽媽好意幫忙拿褲子去洗，卻忘了檢查口袋，結果手機跟著洗壞了。,https://www.facebook.com/tstartel/photos/a.413...
8,100002876464127,劉素珍,"Thursday, May 2, 2019 at 12:34 PM",郁軒，一下叫我洗碗，沒多久又叫我拖地到底是洗碗先還是拖地先,https://www.facebook.com/tstartel/photos/a.413...
9,100024513004893,李李慈,"Thursday, May 2, 2019 at 12:36 PM",筌兒莊 叫我減肥...卻買了滿桌的食物，還準備一堆東西叫我帶回家。😁,https://www.facebook.com/tstartel/photos/a.413...


In [42]:
PostsInformation.to_excel('C:/Users/TL_Yu/Desktop/PostsInfo_TST.xlsx')
PostsComments.to_excel('C:/Users/TL_Yu/Desktop/PostsCmts_TST.xlsx')

# 開發2.0版本

直接在外層抓貼文內容與互動摘要，如果數值沒有改變，就不進去裡面抓留言了

https://m.facebook.com/tstartel/

In [9]:
soup = BeautifulSoup(driver.page_source)

In [12]:
for i in soup.findAll('div',{'class':'_5pcr userContentWrapper'}):
    print(i.text)

台灣之星21 hrs · 再過幾天就是 #母親節 了呢☺ 大家準備好要送給媽媽的禮物了嗎？ #U姐編 雖然還沒晉升阿母階級 但也是很懂阿母的心啦... 對媽媽來說，比起什麼昂貴的禮物 最希望的還是家人健康平安天天幸福💗 所以小編今天也來跟大家做一個約定 請大家今年母親節的時候化身 #護眼大使 不只以身作則照顧自己，也要提醒家人 #不要摸黑滑手機 #用眼30分鐘休息5分鐘 否則 #黃斑部病變上身 #看到黑點就來不及了 好囉！大家都跟姐打勾勾囉🤙 全家人的健康就交給大家守護囉😘 #預祝全天下的媽媽們母親節快樂 #台灣之星正視黃斑部病變 #你不只是用戶更是朋友 #JoinUs See More27427447 Comments2 SharesLikeCommentShareCommentsView 9 more commentsOldestTing-j Huang 母親節快樂💕 · 2h台灣之星 😘 · 2h吳世昌 順吾則生、逆吾則亡，吾乃是轟動武林，驚動萬教萬惡罪魁ㄟ藏鏡人也！小編只有會開遠洋船沒有距離的那家已經釋放出299元吃到飽的母親節優惠方案，因此請小編趕快透露出台灣之星母親節有沒有優惠方案，如果仍守口如瓶的話，我可是要動用滿清十大酷刑對小編你嚴刑逼供了呦！😂😂😂哈哈怕了吧？怕了的話就乖乖的趕緊透露出些許消息出來吧😘😍✌😎。 · 1h台灣之星 小編剛剛午餐已經吃完豆沙包 ~我吾知呀😭。1 · 47m · Edited
台灣之星 added a new photo to the album: 【IGNITE！台灣】慢活尋秘不思議.Yesterday at 12:43 AM · Blue Monday 加 Rainy Day 🌧 這個星期一 #憤輕編 不開心 只能說早上有起床上班 已經是謝天謝地😂... 但是一到下午，瞌睡蟲還是爬上眼皮 只好趕快拿起手機看看照片、看看大海 看看 #瑞芳 #水湳洞 的 #陰陽海 讓心情變美麗😉 大家覺得陰陽海像不像大自然的調色盤呢 希望這張美美的照片 也可以帶給大家美美的心情！ #其實是苦中作樂喇哈哈哈 #台灣之星吃喝玩樂不缺席 See More56656686 Comments8 SharesLikeCommentShare
台灣之星May 4 at 8:57 PM · 【辦門號不吃虧 Ep.3】 這張照片怎麼賴給爸爸❓ 妹妹可以從美

In [ ]:
# 嘗試找文章的ID
# 文章的發布時間
# 順著ID連進去這篇文章
# 爬下文章的按讚數、留言數、分享數..等資訊
# 多少個朋友
# 直播影片數量
# 從留言的頭像可以找到發文者的ID


# 每周台灣之星與競業各自再努力些什麼，可以做為例行性的報告
#[Python] [網路爬蟲] 用 selenium 不用登入就能爬過 Facebook
#https://skylinelimit.blogspot.com/2018/09/python-selenium-facebook.html

In [22]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(3)

# 啟動瀏覽器並登入Facebook
username = driver.find_element_by_id('email')
username.send_keys('61034b001@gms.ndhu.edu.tw')
passwd=driver.find_element_by_id('pass')
passwd.send_keys('19920309')
button=driver.find_element_by_id('loginbutton')
button.click()

In [5]:
# 從時間的元素找出總共有幾篇貼文(會包含到分享到的文章)
print(len(soup.select('div > span > span > a._5pcq')))
posts = []
photos = []
videos = []
for i in soup.select('div > span > span > a._5pcq'):
    url = 'https://www.facebook.com' + i.attrs['href'].split('?',2)[0]
    if bool(re.search('posts', url)) == True:
        posts.append(url)
    elif bool(re.search('photos', url)) == True:
        photos.append(url)
    else :
        videos.append(url)    
print(posts)
print('\n')
print(photos)
print('\n')
print(videos)

46
['https://www.facebook.com/cathayunitedbank/posts/1196903090497027', 'https://www.facebook.com/cathayunitedbank/posts/1198195350367801', 'https://www.facebook.com/cathayunitedbank/posts/1195930853927584', 'https://www.facebook.com/cathayunitedbank/posts/1193308127523190', 'https://www.facebook.com/cathayunitedbank/posts/1193130907540912', 'https://www.facebook.com/cathayunitedbank/posts/1191795174341152', 'https://www.facebook.com/cathayunitedbank/posts/1193302004190469', 'https://www.facebook.com/cathayunitedbank/posts/1191796111007725', 'https://www.facebook.com/cathayunitedbank/posts/1191603801026956', 'https://www.facebook.com/cathayunitedbank/posts/1186570891530247', 'https://www.facebook.com/cathayunitedbank/posts/1186627974857872', 'https://www.facebook.com/cathayunitedbank/posts/1181530582034278', 'https://www.facebook.com/cathayunitedbank/posts/1182106585310011', 'https://www.facebook.com/cathayunitedbank/posts/1182144035306266', 'https://www.facebook.com/cathayunitedbank/p